In [1]:
from cls_VoiceManager import VoiceManager
from cls_ModelManager import Model_Checker
from cls_ModelManager import Agent_Manager
from cls_FileManager import File_Manager
import cls_GeneralTools 
import time
from pathlib import Path

import os


In [2]:
root_path = 'f:\Desktop\AI_TRPG'
fileManager = File_Manager()
rule_name = 'VHS_PROMPT.txt'
story_name = 'THE_SILENCE.txt'
rule = fileManager.read_file(os.path.join(root_path, 'data', 'PROMPT','RULE', rule_name) , print_flag=True)
background = fileManager.read_file(os.path.join(root_path, 'data', 'STORY', story_name) , print_flag=True)
api_key = 'sk-4f981422c9134f0796863adc193c1876'


成功读取文件：f:\Desktop\AI_TRPG\data\PROMPT\RULE\VHS_PROMPT.txt

成功读取文件：f:\Desktop\AI_TRPG\data\STORY\THE_SILENCE.txt


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\32881\AppData\Local\Temp\ipykernel_74652\3921007596.py:1: SyntaxWarning: invalid escape sequence '\D'
  root_path = 'f:\Desktop\AI_TRPG'


In [3]:
am = Agent_Manager(api_key,rule,background)

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import markdown
from tkhtmlview import HTMLLabel
from threading import Thread
import cls_GeneralTools

class StreamDisplayApp:
    def __init__(self, root, agentManager):
        self.root = root
        self.root.title("交互式 Markdown 处理器")
        self.root.geometry("1000x600")
        self.agentManager = agentManager
        self.streaming = False
        self.full_response = ""
        self.history = []
        self.setup_ui()
        self.window_init()

    def setup_ui(self):
        # 配置根窗口网格布局（保持不变）
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_columnconfigure(1, weight=1)
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=0)  # 状态栏行
        self.root.grid_rowconfigure(2, weight=0)  # 复选框行

        # 左侧框架（主容器）
        left_frame = tk.Frame(self.root)
        left_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        left_frame.grid_rowconfigure(0, weight=0)  # 输入区域（固定高度）
        left_frame.grid_rowconfigure(1, weight=1)  # 回复区域（可扩展）
        left_frame.grid_columnconfigure(0, weight=1)  # 单列填充

        # --- 输入区域---
        left_top_frame = tk.Frame(left_frame)
        left_top_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=(10, 5))
        tk.Label(left_top_frame, text="玩家输入：").grid(row=0, column=0, sticky="w")
        self.input_text = scrolledtext.ScrolledText(left_top_frame, wrap=tk.WORD, height=8)
        self.input_text.grid(row=1, column=0, sticky="ew")
        self.input_text.bind("<Return>", self.on_enter)

        # --- 回复显示区域 ---
        left_bottom_frame = tk.Frame(left_frame)
        left_bottom_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=(5, 10))
        left_bottom_frame.grid_rowconfigure(1, weight=1)  # 关键：让回复标签的行可扩展
        left_bottom_frame.grid_columnconfigure(0, weight=1)  # 关键：让列可扩展

        tk.Label(left_bottom_frame, text="主持人：").grid(row=0, column=0, sticky="w")
        self.reply_label = HTMLLabel(left_bottom_frame, html="", wrap=tk.WORD)
        self.reply_label.grid(row=1, column=0, sticky="nsew")  # 使用 sticky="nsew" 填充空间

        # 按钮区域
        button_frame = tk.Frame(self.root)
        button_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=5)
        self.process_button = tk.Button(button_frame, text="下一步", command=self.process_input)
        self.process_button.pack(side=tk.RIGHT)

        # 历史记录区域
        right_frame = tk.Frame(self.root)
        right_frame.grid(row=0, column=1, rowspan=2, sticky="nsew", padx=10, pady=10)
        tk.Label(right_frame, text="历史记录：").grid(row=0, column=0, sticky="w")
        self.history_label = HTMLLabel(right_frame, html="", wrap=tk.WORD)
        self.history_label.grid(row=1, column=0, sticky="nsew")

        # 复选框
        self.checkbox_var = tk.BooleanVar(value=False)
        checkbox = tk.Checkbutton(
            self.root,
            text="文本朗读",
            variable=self.checkbox_var,
            command=self.on_checkbox_change
        )
        checkbox.grid(row=2, column=0, columnspan=2, sticky="w", padx=10, pady=5)
        
        # 状态栏 - 使用grid布局
        self.status_var = tk.StringVar()
        self.status_var.set("准备就绪")
        status_label = tk.Label(self.root, textvariable=self.status_var, bd=1, relief=tk.SUNKEN, anchor=tk.W)
        status_label.grid(row=3, column=0, columnspan=2, sticky="ew")

    def window_init(self):
        # 初始化窗口内容
        reply = self.agentManager.show_background()
        reply_txt = cls_GeneralTools.markdown_to_text_simple(reply)

        initial_input = "# 欢迎使用\n玩家在这里输入..."
        self.input_text.insert(tk.END, initial_input)

        initial_bottom_content = "# 这里是每轮主持人的回复，本场游戏的游戏规则如下：\n" + reply  
        initial_bottom_html = markdown.markdown(initial_bottom_content)
        self.reply_label.set_html(initial_bottom_html)

        initial_right_content = "**这里会显示所有的历史对话记录**"
        initial_right_html = markdown.markdown(initial_right_content)
        self.history_label.set_html(initial_right_html)
        
        self.root.state('zoomed')
        cls_GeneralTools.speak(reply_txt, record_necessary=False, 
                          read_necessary=self.checkbox_var.get(), 
                          output_file='voice')

    # 其他方法保持不变...
    def on_enter(self, event):
        if event.state & 0x1:  # 检查 Shift 键是否被按下
            return
        self.process_input()
        return "break"

    def process_input(self, history=''):
        if self.streaming:
            return
            
        input_content = self.input_text.get("1.0", tk.END).strip()
        if not input_content:
            return
            
        self.streaming = True
        self.full_response = ""
        self.process_button.config(state=tk.DISABLED)
        self.status_var.set("正在获取回复...")
        
        Thread(target=self.fetch_stream_response, args=(input_content, history), daemon=True).start()

    def fetch_stream_response(self, input_content, history):
        try:
            # 使用流式接口获取回复
            response = self.agentManager.stream_reply(content=input_content)
            self.safe_update_status("正在接收回复...")
            
            for chunk in response:
                if not self.streaming:
                    break
                    
                if hasattr(chunk, 'choices') and chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    self.full_response += content
                    self.safe_update_reply(content)
            
            if self.streaming:
                self.finalize_response(history)
                
        except Exception as e:
            self.safe_set_error(f"发生错误: {str(e)}")
        finally:
            self.streaming = False
            self.safe_update_buttons()

    def finalize_response(self, history):
        reply_txt = cls_GeneralTools.markdown_to_text_simple(self.full_response)
        cls_GeneralTools.speak(reply_txt, record_necessary=False, 
                          read_necessary=self.checkbox_var.get(), 
                          output_file='voice')
        
        html_content = markdown.markdown(history + "\n\n" + self.full_response)
        self.safe_update_history(html_content)
        
        self.safe_update_status("回复接收完成")

    def safe_update_reply(self, content):
        self.root.after(0, lambda: self.update_reply(content))

    def safe_update_history(self, html_content):
        self.root.after(0, lambda: self.history_label.set_html(html_content))

    def safe_update_status(self, message):
        self.root.after(0, lambda: self.status_var.set(message))

    def safe_set_error(self, error_msg):
        self.root.after(0, lambda: self.reply_label.set_html(
            f"<div style='color: red; padding: 10px;'>{error_msg}</div>"
        ))

    def safe_update_buttons(self):
        self.root.after(0, lambda: self.process_button.config(state=tk.NORMAL))

    def update_reply(self, content):
        try:
            html_content = markdown.markdown(self.full_response)
            styled_html = f"""
            <div style='
                font-family: Arial, sans-serif;
                font-size: 14px;
                line-height: 1.6;
                padding: 10px;
            '>
                {html_content}
            </div>
            """
            self.reply_label.set_html(styled_html)
            self.reply_label.see(tk.END)
            self.status_var.set(f"接收中... {len(self.full_response)} 字符")
        except Exception as e:
            self.reply_label.set_html(
                f"<div style='color: red; padding: 10px;'>渲染错误: {str(e)}</div>"
            )

    def on_checkbox_change(self):
        print("复选框状态已变化:", self.checkbox_var.get())



In [5]:
if __name__ == "__main__":
    root = tk.Tk()
    
    
    app = StreamDisplayApp(root, am)
    root.mainloop()


成功读取文件：F:\Desktop\AI_TRPG\data\PROMPT\FUNCTION\BEGINNING_PROMPT.txt
